<a href="https://colab.research.google.com/github/alexlinapp/proofLLM/blob/main/finetuning_reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authentication. Make sure you have authentication token as well as access to the gated repo

from huggingface_hub import login

login()

import torch
import transformers

# model_id = "meta-llama/Meta-Llama-3-8B"
# pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype" : torch.bfloat16}, device_map="auto")
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
print(model.lm_head)
#x = model(torch.tensor([[1, 2, 3]]))
print(model)

Linear(in_features=768, out_features=50257, bias=False)
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)

In [ ]:
# config = model.config
# output = model(torch.tensor([[1, 2, 3]]))
# print(output.logits)
# for key, value in model.named_parameters():
#   print(key)
for params in model.parameters():
  params.requires_grad = False
for params in model.lm_head.parameters():
  params.requires_grad = True
for params in model.transformer.h[11].parameters():
  params.requires_grad = True
print("Trainable Parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Trainable Parameters: 45685248


In [ ]:
from datasets import load_dataset

# testing data
dataset = load_dataset("cais/mmlu", "abstract_algebra")
print(dataset.keys())
print(type(dataset["test"]))
print(len(dataset["test"]))



README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

abstract_algebra/test-00000-of-00001.par(…):   0%|          | 0.00/9.96k [00:00<?, ?B/s]

abstract_algebra/validation-00000-of-000(…):   0%|          | 0.00/3.73k [00:00<?, ?B/s]

abstract_algebra/dev-00000-of-00001.parq(…):   0%|          | 0.00/3.45k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

dict_keys(['test', 'validation', 'dev'])
<class 'datasets.arrow_dataset.Dataset'>
100


In [ ]:
dataset = load_dataset("entfane/professor-mathematics")

# each element in the dataset is of type dict
print(dataset["train"][64202])



README.md:   0%|          | 0.00/335 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/61.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/64203 [00:00<?, ? examples/s]

{'question': 'At 30, Anika is 4/3 the age of Maddie. What would be their average age in 15 years?', 'answer': "If Anika is 30 now, in 15 years, she'll be 30+15=<<30+15=45>>45 years old.\nAt 30, Anika is 4/3 the age of Maddie, meaning Maddie is 4/3*30=<<4/3*30=40>>40 years.\nIn 15 years, Maddie will be 40+15=<<40+15=55>>55 years old.\nTheir total age in 15 years will be 55+45=<<55+45=100>>100\nTheir average age in 15 years will be 100/2=<<100/2=50>>50\n#### 50"}


In [ ]:
print(((dataset["train"][:2])))

{'question': ['Let \\( a, b, c \\) be positive real numbers. Prove that\n\n$$\n\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc},\n$$\n\nand that equality occurs if and only if \\( a = b = c = 1 \\).', 'Given real numbers \\( a, b, c \\) and a positive number \\( \\lambda \\) such that the polynomial \\( f(x) = x^3 + a x^2 + b x + c \\) has three real roots \\( x_1, x_2, x_3 \\), and the conditions \\( x_2 - x_1 = \\lambda \\) and \\( x_3 > \\frac{1}{2}(x_1 + x_2) \\) are satisfied, find the maximum value of \\( \\frac{2 a^3 + 27 c - 9 a b}{\\lambda^3} \\).'], 'answer': ["1. Consider the given inequality:\n\n\\[\n\\frac{1}{a(1+b)}+ \\frac{1}{b(1+c)}+ \\frac{1}{c(1+a)} \\geq \\frac{3}{1 + abc}\n\\]\n\nTo simplify, we add \\( \\frac{3}{1 + abc} \\) to both sides. The new inequality becomes:\n\n\\[\n\\frac{1}{a(1+b)} + \\frac{1}{b(1+c)} + \\frac{1}{c(1+a)} + \\frac{3}{1 + abc} \\geq \\frac{6}{1 + abc}\n\\]\n\n2. Let's analyze each term with an added \\( \\frac{1

In [ ]:
from transformers import AutoTokenizer


BASE_CONFIG = {
    "context_length": 512
}
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def format_input(entry):
  instruction_text = (
      f"Below is a question that would like a logical answer. Generate a response that provides a step by step solution."
      f"\n\n### Instruction: {entry['question']}"
  )
  return instruction_text




def pre_process(entry):
  input_text = format_input(entry)
  all_text = input_text + entry['answer']
  return tokenizer(all_text, return_tensors="pt")



In [ ]:
# train_data, val_data, test_data = split_data(dataset)
# train_dataset = dataset["train"][:2]
# type(train_data)
# print(dataset["train"])
print(dataset)
split = dataset["train"].train_test_split(test_size=0.1)
smalldataset = split["train"].select(range(100))
small_train = smalldataset.map(pre_process)
smalldataset_val = split["test"].select(range(100))
small_val = smalldataset_val.map(pre_process)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 64203
    })
})


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1152 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
small_train
small_train.set_format(type="torch", columns=["input_ids"])
small_val.set_format(type="torch", columns=["input_ids"])
small_train[0]['input_ids']

tensor([[21106,   318,   257,  ...,  3467,   737,   198]])

In [ ]:
type(small_train[0])
len(small_train)
small_train[0]['input_ids']
format_input(smalldataset[0])

'Below is a question that would like a logical answer. Generate a response that provides a step by step solution.\n\n### Instruction: Let \\( P(x) \\) be a polynomial with integer coefficients. If \\( n(P) \\) is the number of (distinct) integers \\( k \\) such that \\( P^2(k) = 1 \\), prove that\n\\[ n(P) - \\operatorname{deg}(P) \\leq 2, \\]\nwhere \\( \\operatorname{deg}(P) \\) denotes the degree of the polynomial \\( P \\).'

In [ ]:
from torch.utils.data import DataLoader
import torch
from functools import partial
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
customized_collate_fn = partial(
 custom_collate_fn,
 device=device,
 allowed_max_length=1024
)
torch.manual_seed(123)
train_loader = DataLoader(small_train, batch_size=1, shuffle=True, collate_fn=customized_collate_fn, drop_last=True)
val_loader = DataLoader(small_val, batch_size=6, shuffle=False, collate_fn=customized_collate_fn, drop_last=False)
input, target = next(iter(train_loader))
print(token_ids_to_text(input, tokenizer))

Below is a question that would like a logical answer. Generate a response that provides a step by step solution.

### Instruction: Find the smallest natural number \( n \) such that whenever the set \(\{1, 2, \cdots, n\} \) is divided into any two disjoint subsets, it is always possible to select 3 distinct numbers from one of the subsets where the product of two of them equals the third number.
1. Let's consider the problem which seeks to find the smallest natural number \( n \) such that when the set \(\{1,2, \cdots, n\}\) is arbitrarily divided into two disjoint subsets, it is always possible to find three distinct numbers among them where the product of two of them equals the third one.

2. According to the reference solution, if we start by examining \( n = 95 \) and attempt to partition the set \(\{1,2,\ldots, 95\}\) into two subsets \( A \) and \( B \):

   - For example, one such partition could be:
     \[
     \begin{aligned}
     A &= \{1, 2, 3, 4, 5, 7, 9, 11, 13, 48, 60, 7

In [ ]:
input_text = token_ids_to_text(inputs, tokenizer)
print(generated_text[len(input_text):])



The Pythagorean theorem on the right triangles $ABD$ and $CBD$ gives us:

\[(x^


In [ ]:
model.to(device)
torch.manual_seed(123)
inputs, targets = next(iter(train_loader))
input_ids = text_to_token_ids(format_input(dataset["train"][0]), tokenizer).to(device)
output = generate(model=model, idx=text_to_token_ids(format_input(smalldataset[0]), tokenizer).to(device), context_size=1024, max_new_tokens=30, eos_id=50256)
print("This is output: ", output)
generated_text = token_ids_to_text(output, tokenizer)
print("\n\n==============\n\n")
print(generated_text)

This is output:  tensor([[21106,   318,   257,  1808,   326,   561,   588,   257, 12219,  3280,
            13,  2980,   378,   257,  2882,   326,  3769,   257,  2239,   416,
          2239,  4610,    13,   198,   198, 21017, 46486,    25,   554, 39280,
         28461,  9248,  9738, 47113,  2173,   720,    35,     3,   290,   720,
            36,     3,   389,   319,   262, 18366,   286,   720,  2246,     3,
           290,   720,  6242,     3,  8148,    11,   287,   262,   976,  4571,
            13,  6910,   720, 14529,     3, 36177,    82,  1627,   720,  5222,
             3,   379,   966,   720,    47, 47113,   290,   720, 14529,   796,
         18671, 35307,  1649,   720,    47,     3,   318,   319,   262, 14288,
           286,  1735,   720,  2749, 47113,   905,   326,   720,  6242,   796,
          7125, 35307,   198,   198, 21017, 46486,    25,   554, 39280, 28461,
          9248,  9738, 47113,  2173,   720,    35,     3,   290,   720,    36,
             3,   389,   319,   262

In [ ]:
import time

model.to(device)

start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(
 model.parameters(), lr=0.00005, weight_decay=0.1
)
num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
 model, train_loader, val_loader, optimizer, device,
 num_epochs=num_epochs, eval_freq=5, eval_iter=5,
 start_context=format_input(smalldataset_val[0]), tokenizer=tokenizer
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 2.268, Val loss 2.001
Ep 1 (Step 000005): Train loss 2.366, Val loss 1.955
Ep 1 (Step 000010): Train loss 2.401, Val loss 1.911
Ep 1 (Step 000015): Train loss 1.809, Val loss 1.866
Ep 1 (Step 000020): Train loss 1.966, Val loss 1.827
Ep 1 (Step 000025): Train loss 1.695, Val loss 1.801
Ep 1 (Step 000030): Train loss 1.872, Val loss 1.779
Ep 1 (Step 000035): Train loss 2.223, Val loss 1.762
Ep 1 (Step 000040): Train loss 1.793, Val loss 1.752
Ep 1 (Step 000045): Train loss 2.204, Val loss 1.743
Ep 1 (Step 000050): Train loss 2.184, Val loss 1.734
Ep 1 (Step 000055): Train loss 2.068, Val loss 1.727
Ep 1 (Step 000060): Train loss 1.695, Val loss 1.720
Ep 1 (Step 000065): Train loss 1.844, Val loss 1.713
Ep 1 (Step 000070): Train loss 1.856, Val loss 1.706
Ep 1 (Step 000075): Train loss 1.741, Val loss 1.698
Ep 1 (Step 000080): Train loss 1.860, Val loss 1.692
Ep 1 (Step 000085): Train loss 1.724, Val loss 1.686
Ep 1 (Step 000090): Train loss 1.865, Val loss

In [ ]:
input_text = text_to_token_ids(format_input(smalldataset[0]), tokenizer)
output_ids = generate(model=model, idx=input_text.to(device), context_size=1024, max_new_tokens=100, eos_id=50256)

In [ ]:
print(token_ids_to_text(output_ids, tokenizer)[len(format_input(smalldataset[0])):])
print("\n\n============\n\n")
print(smalldataset[0])



### Instruction: Let \( P(x) \) be a polynomial with integer coefficients.

### Instruction: Let \( P(x) \) be a polynomial with integer coefficients.

### Instruction: Let \( P(x) \) be a polynomial with integer coefficients.
### Instruction: Let \( P(x) \) be a polynomial with integer coefficients.

### Instruction: Let \( P(x) \ be




{'question': 'Let \\( P(x) \\) be a polynomial with integer coefficients. If \\( n(P) \\) is the number of (distinct) integers \\( k \\) such that \\( P^2(k) = 1 \\), prove that\n\\[ n(P) - \\operatorname{deg}(P) \\leq 2, \\]\nwhere \\( \\operatorname{deg}(P) \\) denotes the degree of the polynomial \\( P \\).', 'answer': '\n1. **Understanding the problem**:\n   - We are given a polynomial \\( P(x) \\) with integer coefficients.\n   - Define \\( n(P) \\) as the number of distinct integers \\( k \\) such that \\( P^2(k) = 1 \\).\n   - We need to prove that \\( n(P) - \\operatorname{deg}(P) \\leq 2 \\), where \\( \\operatorname{deg}(P) \\) is the degr

In [ ]:
def custom_collate_fn(batch, pad_token_id=50256,
                      ignore_index=-100,
                      allowed_max_length=None,
                      device="cpu"):
  batch_max_length = max(item['input_ids'].shape[1] + 1 for item in batch)
  if allowed_max_length is not None:
    batch_max_length = min(batch_max_length, allowed_max_length+1)


  inputs_lst, targets_lst = [], []
  for item in batch:
    new_item = item['input_ids'][0].clone()
    pad_length = batch_max_length - new_item.shape[0]
    padding = torch.full((max(0, pad_length),), pad_token_id)
    padded = torch.cat([new_item[:batch_max_length], padding], dim=0)
    inputs = padded[:-1].clone()
    targets = padded[1:].clone()
    targets[targets.shape[0]-1] = pad_token_id
    mask = targets == pad_token_id
    indices = torch.nonzero(mask).squeeze()
    if indices.numel() > 1:
      targets[indices[1:]] = ignore_index


    inputs_lst.append(inputs)
    targets_lst.append(targets)
  inputs_tensor = torch.stack(inputs_lst, dim=0).to(device)
  targets_tensor = torch.stack(targets_lst, dim=0).to(device)
  return inputs_tensor, targets_tensor


In [ ]:
def calc_accuracy_batch(input_batch, target_batch, model, device):
  input_batch, target_batch = input_batch.to(device), target_batch.to(device)
  logits = model(input_batch).logits
  preds = torch.argmax(logits, dim=-1)

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch).logits
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches



def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            output = model(idx_cond)
            if isinstance(output, transformers.utils.ModelOutput):
                logits = output.logits
            else:
                logits = output
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float('-inf')).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx


def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()  # Calculate loss gradients
            optimizer.step()  # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        # generate_and_print_sample(
        #     model, tokenizer, device, start_context
        # )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()


def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor


def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())
